In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

from prefect_tasks.bronze import _PROJECT_ROOT

In [ ]:
BRONZE_PATH = _PROJECT_ROOT / "data" / "bronze" / "raw_impressions.parquet"
conn = duckdb.connect()

raw_impressions = conn.execute(f"""
    SELECT * FROM '{BRONZE_PATH}'             
""")

raw_impressions.fetch_df_chunk()